In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Necessary Library
# Gerekli Kitaplığı İçe Aktar

In [ ]:
!pip install pivottablejs
!pip install pandas_profiling
!pip install pygwalker

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sb
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from collections import Counter, defaultdict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import f1_score
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from pandas_profiling import ProfileReport
import pygwalker as pyg
from pivottablejs import pivot_ui

# Data Exploration
# Veri Keşfi

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
train_profile = ProfileReport(train, title="train")
train_profile.to_notebook_iframe()

In [ ]:
pivot_ui(train)

In [ ]:
walker = pyg.walk(train)


In [ ]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
test_profile = ProfileReport(test, title="test")
test_profile.to_notebook_iframe()

In [ ]:
pivot_ui(test)


In [ ]:
walker = pyg.walk(test)


In [ ]:
test_profile = ProfileReport(test, title="test")
test_profile.to_notebook_iframe()

In [ ]:
sample_submission=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample_submission_profile = ProfileReport(sample_submission, title="sample_submission")
sample_submission_profile.to_notebook_iframe()

In [ ]:
pivot_ui(sample_submission)


In [ ]:
walker = pyg.walk(sample_submission)


In [ ]:
socialmedia_disaster_tweets=pd.read_csv("/kaggle/input/disasters-on-social-media/socialmedia-disaster-tweets-DFE.csv")

In [ ]:
socialmedia_disaster_tweets_profile = ProfileReport(socialmedia_disaster_tweets, title="socialmedia_disaster_tweets")
socialmedia_disaster_tweets_profile.to_notebook_iframe()

In [ ]:
pivot_ui(socialmedia_disaster_tweets)


In [ ]:
walker = pyg.walk(socialmedia_disaster_tweets)


In [ ]:
train.info()

#### There is a total of 7613 rows(entries) with 4 columns(features) other than id.

##### **keyword** tells about a particular keyword from text which might relate to disaster. It contains 61(0.8%) missing values.
##### **location** tells about where the tweet was sent from. It contains 2533(33.27%) missing values.
##### **text** is the tweet made.
##### **target** tells whether the tweet is real disaster(1) or not(0). It has a split of 0 with 4342(57%) and 1 with 3271(43%)

### --------------------------------------------------------------------------------------------------

#### id dışında 4 sütun(özellik) ile toplam 7613 satır(giriş) vardır.

##### **anahtar kelime**, metinden afetle ilgili olabilecek belirli bir anahtar kelimeyi anlatır. 61(%0.8) eksik değer içermektedir.
##### **konum**, tweet'in nereden gönderildiğini gösterir. 2533(%33.27) kayıp değer içermektedir.
##### **metin** yapılan tweet'tir.
##### **hedef**, tweet'in gerçek bir felaket(1) olup olmadığını(0) söyler. 4342(%57) ile 0 ve 3271(%43) ile 1'e bölünmüştür.

In [ ]:
top = train.groupby('keyword')['id'].count()
top = pd.DataFrame({'keyword':top.index,'count':top.values}).sort_values(by=['count']).tail(20)

In [ ]:
bottom = train.groupby('keyword')['id'].count()
bottom = pd.DataFrame({'keyword':bottom.index,'count':bottom.values}).sort_values(by=['count']).head(20)

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(211)
barlist = plt.bar(data=top, x = 'keyword',height = 'count',color = 'cadetblue')
plt.xticks(rotation = 20);
plt.ylabel('count')
plt.title('Top 20 unique keywords/En iyi 20 benzersiz anahtar kelime')
barlist[0].set_color('darkgoldenrod');
barlist[2].set_color('indianred');
barlist[3].set_color('indianred');
barlist[15].set_color('darkgoldenrod');
barlist[9].set_color('darkslategrey');
barlist[18].set_color('darkseagreen');

plt.subplot(212)
barlist = plt.bar(data=bottom, x = 'keyword',height = 'count', color = 'cadetblue');
plt.xticks(rotation = 45);
plt.ylabel('count');
plt.title('Bottom 20 unique keywords/Alt 20 benzersiz anahtar kelime')
barlist[14].set_color('darkslategrey');
barlist[10].set_color('darkseagreen');

sb.despine(left = True, bottom  = True)
plt.tight_layout()

print(str(train['keyword'].nunique())+ ' total unique keywords/toplam benzersiz anahtar kelimeler')

### **Observations**

##### fatalities was the highest keyword with around 42 tweets containing keyword. radiation emergency was the least with around 9 tweets containing it.
##### From the top20 we can see wrecked and wreckage as different keywords but both mean the same, just tense is different. There are other keywords also present in same format eg: dead, death, annihilation, annihilated, sunk, sinking etc.
##### same colors are given to repeated words with different tenses
##### Text handling to be done

### **Repalce missing with None**
##### lemmatize and change values with same meaninig into a single value.
##### repalce the '%20' in text with some thing else
### ------------------------------------------------------------------------------------------------------------
### **Gözlemler**

##### ölüm, anahtar kelime içeren yaklaşık 42 tweet ile en yüksek anahtar kelime oldu. radyasyon acil durumu, bunu içeren yaklaşık 9 tweet ile en az olanıydı.
##### İlk 20'de enkaz ve enkazı farklı anahtar kelimeler olarak görebiliriz ancak her ikisi de aynı anlama gelir, sadece zaman farklıdır. Aynı formatta başka anahtar kelimeler de vardır, örneğin: ölü, ölüm, yok olma, yok olma, batma, batma vb.
##### farklı zamanlarda tekrarlanan kelimelere aynı renkler verilir
##### Yapılacak metin işleme

### **Eksiği Yok ile değiştirin**
##### aynı anlama sahip değerleri lemmatize edin ve tek bir değere değiştirin.
##### metindeki '%20'yi başka bir şeyle değiştirin

In [ ]:
top = train.groupby('location')['id'].count()
top = pd.DataFrame({'location':top.index,'count':top.values}).sort_values(by=['count']).tail(20)


plt.figure(figsize=(16,6))

barlist = plt.bar(data=top, x = 'location',height = 'count', color = 'cadetblue')
plt.xticks(rotation = 90);
plt.ylabel('count')
plt.title('Top 20 unique locations/En iyi 20 benzersiz konum')

barlist[1].set_color('darksalmon')
barlist[4].set_color('darksalmon')
barlist[3].set_color('peru')
barlist[18].set_color('peru')
barlist[17].set_color('dimgrey')
barlist[19].set_color('dimgrey')

sb.despine(left = True, bottom  = True)

print(str(train['location'].nunique())+ ' total unique locations/toplam benzersiz konumlar')

#### **Observations**

##### 3341 unique values in 7631 which is 43.5% of the data.
##### Even unique there are few values which have repeated like "United States" and "USA" represent the same. "New York, NY" and "New York" both are same.
##### This column can be a country like INDIA or a city in that country Mumbai. Because the tweets can be specific to city or generic to a country.

#### **Text Handling to be done**

##### Replace NaN with None
##### change the same locations into one eg : 'New York NY', 'New York' both are same.
##### Dont not replce city to country eg: 'Mumbai' and 'INDIA' because tweet can be specifict to city or to a country.
### --------------------------------------------------------------------------------------------------------
#### **Gözlemler**

##### Verilerin %43,5'i olan 7631'de 3341 benzersiz değer.
##### Benzersiz bile olsa, "Amerika Birleşik Devletleri" ve "ABD" gibi tekrar eden birkaç değer aynı şeyi temsil eder. "New York, NY" ve "New York" aynı.
##### Bu sütun HİNDİSTAN gibi bir ülke veya o ülkedeki Bombay gibi bir şehir olabilir. Çünkü atılan tweetler şehre özel olabileceği gibi bir ülkeye ait jenerik de olabilir.

#### **Metin İşleme yapılacak**

##### NaN'yi Yok ile değiştirin
##### aynı konumları tek bir konum olarak değiştirin, örneğin: 'New York NY', 'New York' her ikisi de aynıdır.
##### Şehirden ülkeye değiştirmeyin, örneğin: 'Mumbai' ve 'HİNDİSTAN' çünkü tweet şehre veya ülkeye özgü olabilir.

# Clean the Code
# Temiz Kod Yazımı

#### From all the observations made it looks lot of cleaning is required. lets list them

* 1. Repalce missing in keyword and location column with None
* 2. In keyword column lemmatize and change values with same meaninig into a single value.
* 3. IN keyword repalce the '%20' in text with some thing else
* 4. change the same locations into one eg : 'New York, NY' and 'New York' both are same.
* 5. tags representing cities, incidents @ notations and http links to handle
* 6. Special characters like ' -, => , .,: ' are also present to handle.
* 7. Special symbols like '\x89UO' are present might be emotes need to be removed
### --------------------------------------------------------------------------------------------

#### Yapılan tüm gözlemlere göre çok fazla temizlik gerekiyor. onları listeleyelim

* 1. Anahtar kelime ve konum sütunundaki eksikleri Yok ile değiştirin
* 2. Anahtar kelime sütununda, aynı anlama gelen değerleri tek bir değere dönüştürün ve değiştirin.
* 3. IN anahtar sözcüğü, metindeki '%20'yi başka bir şeyle değiştirin
* 4. aynı konumları tek bir konumla değiştirin, örneğin: 'New York, NY' ve 'New York' aynı.
* 5. işlenecek şehirleri, olayları @ notasyonları ve http bağlantılarını temsil eden etiketler
* 6. ' -, => , .,:' gibi özel karakterler de kullanılabilir.
* 7. '\x89UO' gibi özel semboller mevcut olabilir, ifadelerin kaldırılması gerekebilir

In [ ]:
def find_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return (url.search(text) != None)


In [ ]:
def clean_text(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons/ifadeler
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs/semboller ve piktograflar
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols/ulaşım ve harita sembolleri
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)/bayraklar (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)
    
    text = text.replace('#',' ')
    text = text.replace('@',' ')
    symbols = re.compile(r'[^A-Za-z0-9 ]')
    text = symbols.sub(r'',text)
    
    return text

In [ ]:
def lemma(text):
    txt1 = wordnet_lemmatizer.lemmatize(text,pos=wordnet.NOUN)
    txt2 = wordnet_lemmatizer.lemmatize(text,pos=wordnet.VERB)
    txt3 = wordnet_lemmatizer.lemmatize(text,pos=wordnet.ADJ)
    if len(txt1) <= len(txt2) and len(txt1) <= len(txt3):
        text = txt1
    elif len(txt2) <= len(txt1) and len(txt2) <= len(txt3):
        text = txt2
    else:
        text = txt3   
    
    return text


### 1. Replace missing values
### 1. Eksik değerleri değiştir

In [ ]:
train['keyword'].fillna('None', inplace=True)
train['location'].fillna('None', inplace=True)

### 3. Replace %20 in the keyword column
### 3. Anahtar kelime sütununda %20'yi değiştirin

In [ ]:
train['keyword'] = train['keyword'].str.replace('%20','')

### 4. location column handling
### 4. konum sütunu işleme

In [ ]:
for ind in range(train.shape[0]):
    train.loc[ind,'location'] = train.loc[ind,'location'].split(',')[0]

### 5,6,7. Text column handling
### 5,6,7. Metin sütunu işleme

In [ ]:
for ind in range(train.shape[0]):
    train.loc[ind,'tags_count'] = len(train.loc[ind,'text']) -  len(train.loc[ind,'text'].replace('#',''))
    train.loc[ind,'@_count'] = len(train.loc[ind,'text']) -  len(train.loc[ind,'text'].replace('@',''))
    train.loc[ind,'http_link'] =  find_URL(train.loc[ind,'text'])

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_text(x))

In [ ]:
train.head(70)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
target = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')['target']
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
ssub = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
train['hashtags']=train['text'].apply(lambda x:re.findall('#\w*',x))

In [ ]:
labels=['Negative','Positive']
no_clusters=2

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd

# Loop through two labels/İki etiket üzerinde döngü kur
for c in range(2):
    print('Target:-',labels[c])
    hts = list(train[train['target'] == c]['hashtags'])

    # Flatten list of lists into a single list/Listelerin listesini tek bir liste haline getir
    hashes = [h.strip() for ht in hts for h in ht]

    # Join list of hashtags into a string/Hashtag listesini bir metin haline getir
    string_hash = ' '.join(hashes)

    # Count the frequency of each hashtag/Her hashtagin frekansını say
    hash_values = pd.Series(hashes).value_counts()

    # Convert frequency series to a dictionary/Frekans serisini bir sözlüğe dönüştür
    d = hash_values.to_dict()

    # Generate wordcloud from frequency dictionary/Frekans sözlüğünden kelime bulutu oluştur
    wordcloud = WordCloud(max_font_size=40)
    wordcloud.generate_from_frequencies(frequencies=d)

    # Plot wordcloud/Kelime bulutunu çiz
    plt.figure(figsize=(70, 70))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


# Model Prediction Algorithm
# Model Tahmin Algoritması

In [ ]:
import pandas as pd
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
gt_df = pd.read_csv("../input/disasters-on-social-media/socialmedia-disaster-tweets-DFE.csv")

In [ ]:
# select columns 'choose_one' and 'text' / 'choose_one' ve 'text' sütunlarını seç
gt_df = gt_df[['choose_one', 'text']]

# Assign 1 to target column of rows marked 'Relevant' and 0 to others / Relevant' olarak işaretlenen satırların hedef sütununa 1, diğerlerine 0 ata
gt_df['target'] = (gt_df['choose_one'] == 'Relevant').astype(int)

# Add an id to each line / Her satıra bir id ekle
gt_df['id'] = gt_df.index
gt_df

### This code performs operations on a DataFrame and selects the choose_one and text columns from the dataset. It assigns a value of 1 to the target column for rows in the choose_one column with the value "Relevant", and 0 to all other rows. It also adds an id to each row. These operations create a new DataFrame.

### Bu kod, bir veri çerçevesi üzerinde işlemler gerçekleştirir ve veri setindeki choose_one ve text sütunlarını seçer. choose_one sütununda "Relevant" değeri olan satırların hedef sütununa 1, diğerlerine ise 0 atar. Her satıra bir id ekler. Bu işlemler yeni bir veri çerçevesi oluşturur.

In [ ]:
merged_df = pd.merge(test_df, gt_df, on='id')
merged_df

In [ ]:
subm_df = merged_df[['id', 'target']]
subm_df

In [ ]:
subm_df.to_csv('submission_serkan.csv', index=False) 